In [1]:
import pandas as pd
pd.reset_option('display.max_rows')

In [6]:
data = pd.read_excel("data/final/Shiv Jayanti Marathon Run.xlsx")
data['participants'] = data['Participant Details'].apply(lambda x: x.split(','))
flatdata = pd.DataFrame([( index, value) for ( index, values) 
                         in enumerate(data['participants']) for value in values], 
                             columns = [ 'index', 'participant_details']).set_index( 'index' ) 

df = data.drop( 'participants', axis = 1 ).join( flatdata ) 
df.to_excel("data/flatten_participants.xlsx")

In [5]:
data.count()

Timestamp                    147
Name                         152
Email                        119
Flat#                        144
Contact Number (Whatsapp)    148
Participant Details          152
Payment snapshot             152
Fitness Disclaimer           119
participants                 152
dtype: int64

In [7]:
from functools import reduce
import re
cleaned_participants = pd.read_excel("data/flatten_participants.xlsx")
substrings = ["km", "\n" "run"]
cleaned_participants['participant_details'].apply(lambda x: reduce(lambda t, s: s.replace(t, ''), substrings, str(x)))

cleaned_participants['participant_details'] = cleaned_participants['participant_details'].apply(lambda x: str(x).split("-"))
cleaned_participants['name'] = cleaned_participants['participant_details'].apply(lambda x: x[0].replace('\n', '').strip())
cleaned_participants['flat'] = cleaned_participants['participant_details'].apply(lambda x: x[1])
cleaned_participants['age'] = cleaned_participants['participant_details'].apply(lambda x: int("".join([ele for ele in x[3] if ele.isdigit()])))

cleaned_participants['run_category'] = cleaned_participants['participant_details'].apply(lambda x: int("".join([ele for ele in x[3] if ele.isdigit()])))
final_participants = cleaned_participants[['name', 'age', 'run_category']].sort_values(['run_category', 'name'],ascending=[False, True])

final_participants

,name,age,run_category
105,Ajit Patil,10,10
35,Amit Thumbar,10,10
66,Gajender Kumar,10,10
73,Mannu,10,10
203,Mihika,10,10
...,...,...,...
84,Virenkumar,3,3
38,Yogita,3,3
23,gaurav,3,3
113,subhra,3,3


In [4]:
cleaned_participants[cleaned_participants['participant_details'].map(len) < 4]

,Unnamed: 0,Timestamp,Name,Email,Flat#,Contact Number (Whatsapp),Participant Details,Payment snapshot,Fitness Disclaimer,participant_details,name,flat,age,run_category


In [8]:
final_participants.groupby(['run_category']).agg({'name': 'count'})

,name
run_category,
3,148
5,63
10,19


In [9]:
final_participants.to_excel("data/final/final_marathon_participants.xlsx")

In [96]:
pip install python-pptx

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.8 MB 6.3 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 7.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --------------------------- ------------ 1.8/2.6 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import io
import copy
def duplicate_slide(pres, template, name, bib):

    #Add a new slide
    copied_slide = pres.slides.add_slide(template.slide_layout)

    #delete existing shapes
    for shp in copied_slide.shapes:
        copied_slide.shapes.element.remove(shp.element)
    
    #perform deep copy of the templates shapes

    for shp in template.shapes:
        copy_shp = copy.deepcopy(shp)

        if "Picture" in copy_shp.name:
            img = io.BytesIO(shp.image.blob)
            copied_slide.shapes.add_picture(image_file=img,
                                            left = shp.left,
                                            top = shp.top,
                                            width = shp.width,
                                            height = shp.height)
        elif hasattr(copy_shp, "text"):
            text = copy_shp.text

            if text.lower() in ['<< bib >>']:
                copy_shp.text_frame.paragraphs[0].runs[0].text = str(bib)
            
            if text.lower() in ['<< name >>']:
                copy_shp.text_frame.paragraphs[0].runs[0].text = str(name)
            newel = copy_shp.element
            copied_slide.shapes._spTree.insert_element_before(newel)
        
        else:
            newel = copy_shp.element
            copied_slide.shapes._spTree.insert_element_before(newel)
        
    return pres



In [11]:
from pptx import Presentation

ppt = Presentation("templates_bibs.pptx")
gen_ppt = Presentation("templates_bibs.pptx")
ten_template = ppt.slides[0]
five_template = ppt.slides[1]
three_template = ppt.slides[2]


In [12]:
seed_10, seed_5, seed_3 = 1000, 500, 300

for row in final_participants.iterrows():
    name = row[1]["name"]
    if row[1]['run_category'] == 10:
        template = ten_template
        seed_10 += 1
        gen_ppt = duplicate_slide(gen_ppt, template, name, seed_10)
    elif row[1]["run_category"] == 5:
        template = five_template
        seed_5 += 1
        gen_ppt = duplicate_slide(gen_ppt, template, name, seed_5)
    else:
        template = three_template
        seed_3 += 1
        gen_ppt = duplicate_slide(gen_ppt, template, name, seed_3)



In [13]:
gen_ppt.save('generated_bib.pptx')

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile\__init__.py:1612: UserWarning: Duplicate name: 'ppt/slideLayouts/slideLayout1.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile\__init__.py:1612: UserWarning: Duplicate name: 'ppt/slideLayouts/_rels/slideLayout1.xml.rels'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile\__init__.py:1612: UserWarning: Duplicate name: 'ppt/slideMasters/slideMaster1.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile\__init__.py:1612: UserWarning: Duplicate name: 'ppt/slideMasters/_rels/slideMaster1.xml.rels'
  return self._open_to_writ